In [2]:
!pip install nrclex
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import pandas as pd

new_song = pd.read_csv('/content/drive/MyDrive/ISR/datasets/preprocessed_datasets/preprocessed_songs_chunk_0.csv')
new_song.head()

,artist,title,year,lyrics
0,Cam'ron,Killa Cam,2004,"[chorus: opera steve & cam'ron] killa cam, kil..."
1,JAY-Z,Can I Live,1996,"[produced by irv gotti] [intro] yeah, hah, ye..."
2,Fabolous,Forgive Me Father,2003,maybe cause i'm eatin and these bastards fiend...
3,Cam'ron,Down and Out,2004,[produced by kanye west and brian miller] [in...
4,Lil Wayne,Fly In,2005,"[intro] so they ask me ""young boy what you gon..."


In [ ]:
import pandas as pd
from nrclex import NRCLex
from collections import defaultdict

# Read the input CSV file
input_file = '/content/drive/MyDrive/ISR/datasets/preprocessed_datasets/preprocessed_songs_chunk_0.csv'
lyrics_df = pd.read_csv(input_file)

# Perform the operations and store the results
results = []
formatted_score = defaultdict(int)

for i, row in lyrics_df.iterrows():
    lyrics = row['lyrics']
    artist = row['artist']
    title = row['title']

    emotion = NRCLex(lyrics)
    raw_score = emotion.raw_emotion_scores

    if len(raw_score) > 1:
        mean_score = sum(raw_score.values()) / len(raw_score)
        std_dev = (sum((x - mean_score) ** 2 for x in raw_score.values()) / len(raw_score)) ** 0.5
    else:
        std_dev = 1  # Set std_dev to a non-zero value to avoid division by zero

    if std_dev == 0:
        normalized_score = {emotion: 0 for emotion, score in raw_score.items()}
    else:
        normalized_score = {emotion: (score - mean_score) / std_dev for emotion, score in raw_score.items()}

    if normalized_score:  # Check if the dictionary is not empty
        max_score = max(normalized_score.values())
        min_score = min(normalized_score.values())
    else:
        max_score, min_score = 0, 0

    if max_score != min_score:
        normalized_score = {emotion: (score - min_score) / (max_score - min_score) for emotion, score in normalized_score.items()}
    else:
        normalized_score = {emotion: 0 for emotion, score in raw_score.items()}


    formatted_score = {emotion: format(score, '.4f') for emotion, score in normalized_score.items()}


    results.append({
        'anger': formatted_score.get('anger', 0),
        'anticipation': formatted_score.get('anticipation', 0),
        'disgust': formatted_score.get('disgust', 0),
        'fear': formatted_score.get('fear', 0),
        'joy': formatted_score.get('joy', 0),
        'positive': formatted_score.get('positive', 0),
        'surprise': formatted_score.get('surprise', 0),
        'trust': formatted_score.get('trust', 0),
        'negative': formatted_score.get('negative', 0)
})

results_df = pd.DataFrame(results)

# Save the results to a new CSV file
output_file = '/content/drive/MyDrive/ISR/datasets/preprocessed_datasets/normalized_sentiments_dataset_0.csv'
results_df.to_csv(output_file, index=False)
